In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
raw = pd.read_excel(
  '../../data/reference/qualifications_2021census.xlsx',
  sheet_name='quals_con'
).rename(
    columns={'ONSConstID': 'constituency_code',
            'ConstituencyName': 'constituency_name',
            'variables': 'variable',
            'Con_num': 'value',
            'Con_pc': 'percent'}).set_index([
            'constituency_code', 
            'constituency_name']).drop(
    columns={'NatComparator', 
            'RegNationID',
            'RegNationName',
            'RN_pc', 
            'Nat_pc', 
            'ranking_total',
            'rank'})

raw.columns = raw.columns.str.strip()
raw

variable  \
constituency_code constituency_name                                            
E14000530         Aldershot                                No qualifications   
                  Aldershot                               1 to 4 GCSE passes   
                  Aldershot                            5 or more GCSE passes   
                  Aldershot                                   Apprenticeship   
                  Aldershot                               2 or more A levels   
...                                                                      ...   
W07000080         Cardiff South and Penarth            5 or more GCSE passes   
                  Cardiff South and Penarth                   Apprenticeship   
                  Cardiff South and Penarth               2 or more A levels   
                  Cardiff South and Penarth  Higher education qualifications   
                  Cardiff South and Penarth             Other qualifications   

                                                                      groups  \
constituency_code constituency_name                                            
E14000530         Aldershot                                No qualifications   
                  Aldershot                                  1 or more GCSEs   
                  Aldershot                                  1 or more GCSEs   
                  Aldershot                                   Apprenticeship   
                  Aldershot                               2 or more A levels   
...                                                                      ...   
W07000080         Cardiff South and Penarth                  1 or more GCSEs   
                  Cardiff South and Penarth                   Apprenticeship   
                  Cardiff South and Penarth               2 or more A levels   
                  Cardiff South and Penarth  Higher education qualifications   
                  Cardiff South and Penarth             Other qualifications   

                                             value   percent  
constituency_code constituency_name                           
E14000530         Aldershot                  16148  0.178468  
                  Aldershot                  10772  0.119053  
                  Aldershot                  12998  0.143654  
                  Aldershot                   5148  0.056896  
                  Aldershot                  15712  0.173650  
...                                            ...       ...  
W07000080         Cardiff South and Penarth  11193  0.122620  
                  Cardiff South and Penarth   4142  0.045376  
                  Cardiff South and Penarth  13653  0.149569  
                  Cardiff South and Penarth  34262  0.375342  
                  Cardiff South and Penarth   2572  0.028176  

[4011 rows x 4 columns]

In [3]:
os.makedirs('../../data/processed/yff/', exist_ok=True)
raw.to_csv('../../data/processed/yff/census_qualifications.csv')

### Census 2021 - Qualifications by ethnic group, local authority breakdown

In [4]:


la_reference_data = pd.read_csv('../../data/reference/local_authorities.csv', usecols=['Local Authority Code', 'Local Authority Name'])

In [5]:
census_ethnic_group = pd.read_csv(
    '../../data/reference/census-qualifications-ethnic-group.csv', index_col=0
)

la_reference_data = pd.read_csv('../../data/reference/local_authorities.csv', usecols=['Local Authority Code', 'Local Authority Name'])

census_ethnic_group = census_ethnic_group.merge(
    la_reference_data, 
    how='left',
    left_on= census_ethnic_group.index,
    right_on= 'Local Authority Name'
    )
census_ethnic_group = census_ethnic_group.rename(columns= { 'Local Authority Code': 'local_authority_code', 
                                                           'Local Authority Name': 'local_authority_name'})

census_ethnic_group = census_ethnic_group.set_index(['local_authority_code', 'local_authority_name'])
census_ethnic_group

Total            Total.1  \
local_authority_code local_authority_name                              
NaN                  local_authority        Total  No qualifications   
                     NaN                      NaN                NaN   
E06000001            Hartlepool             74688              17286   
E06000002            Middlesbrough         113935              27281   
E06000003            Redcar and Cleveland  112344              24731   
...                                           ...                ...   
NaN                  Torfaen                75111              16631   
                     Monmouthshire          78237              12316   
                     Newport               127419              27688   
                     Powys                 112536              21316   
                     Merthyr Tydfil         47679              12832   

                                                                          Total.2  \
local_authority_code local_authority_name                                           
NaN                  local_authority       Level 1 and entry level qualifications   
                     NaN                                                      NaN   
E06000001            Hartlepool                                              7076   
E06000002            Middlesbrough                                          11221   
E06000003            Redcar and Cleveland                                   10986   
...                                                                           ...   
NaN                  Torfaen                                                 7697   
                     Monmouthshire                                           6461   
                     Newport                                                11741   
                     Powys                                                   9694   
                     Merthyr Tydfil                                          4556   

                                                          Total.3  \
local_authority_code local_authority_name                           
NaN                  local_authority       Level 2 qualifications   
                     NaN                                      NaN   
E06000001            Hartlepool                             10963   
E06000002            Middlesbrough                          15271   
E06000003            Redcar and Cleveland                   15663   
...                                                           ...   
NaN                  Torfaen                                11457   
                     Monmouthshire                          10588   
                     Newport                                18186   
                     Powys                                  16747   
                     Merthyr Tydfil                          6911   

                                                  Total.4  \
local_authority_code local_authority_name                   
NaN                  local_authority       Apprenticeship   
                     NaN                              NaN   
E06000001            Hartlepool                      5205   
E06000002            Middlesbrough                   7040   
E06000003            Redcar and Cleveland            8693   
...                                                   ...   
NaN                  Torfaen                         4313   
                     Monmouthshire                   3958   
                     Newport                         6498   
                     Powys                           6869   
                     Merthyr Tydfil                  2716   

                                                          Total.5  \
local_authority_code local_authority_name                           
NaN                  local_authority       Level 3 qualifications   
                     NaN                                      NaN   
E06000001            Hartlepool  

In [6]:

census_ethnic_group.columns = census_ethnic_group.columns.str.lower()

census_ethnic_group = census_ethnic_group.iloc[2:]

data = pd.MultiIndex.from_product([['total','asian', 'black_british', 'mixed_multiple', 'white', 'other'],
                                     ['total','no_qualification','level_1', 'level_2', 'apprenticeship','level_3', 'level_4', 'other']])

census_ethnic_group.columns = data
census_ethnic_group.reset_index().to_csv('../../data/processed/census/qualifications_local_authority.csv', index=False)
census_ethnic_group

total                           \
                                            total no_qualification level_1   
local_authority_code local_authority_name                                    
E06000001            Hartlepool             74688            17286    7076   
E06000002            Middlesbrough         113935            27281   11221   
E06000003            Redcar and Cleveland  112344            24731   10986   
E06000004            Stockton-on-Tees      158016            29441   15327   
E06000005            Darlington             88106            16971    8276   
...                                           ...              ...     ...   
NaN                  Torfaen                75111            16631    7697   
                     Monmouthshire          78237            12316    6461   
                     Newport               127419            27688   11741   
                     Powys                 112536            21316    9694   
                     Merthyr Tydfil         47679            12832    4556   

                                                                          \
                                          level_2 apprenticeship level_3   
local_authority_code local_authority_name                                  
E06000001            Hartlepool             10963           5205   13778   
E06000002            Middlesbrough          15271           7040   19974   
E06000003            Redcar and Cleveland   15663           8693   21300   
E06000004            Stockton-on-Tees       23006          10567   29004   
E06000005            Darlington             12733           6216   16140   
...                                           ...            ...     ...   
NaN                  Torfaen                11457           4313   12736   
                     Monmouthshire          10588           3958   12172   
                     Newport                18186           6498   19976   
                     Powys                  16747           6869   19124   
                     Merthyr Tydfil          6911           2716    7454   

                                                         asian  \
                                          level_4 other  total   
local_authority_code local_authority_name                        
E06000001            Hartlepool             18525  1855   1182   
E06000002            Middlesbrough          30125  3023  11159   
E06000003            Redcar and Cleveland   28003  2968    902   
E06000004            Stockton-on-Tees       46648  4023   6615   
E06000005            Darlington             25517  2253   2231   
...                                           ...   ...    ...   
NaN                  Torfaen                20125  2152   1008   
                     Monmouthshire          30809  1933    966   
                     Newport                39544  3786   8737   
                     Powys                  35849  2937   1045   
                     Merthyr Tydfil         11911  1299    685   

                                                            ...   white        \
                                          no_qualification  ... level_4 other   
local_authority_code local_authority_name                   ...                 
E06000001            Hartlepool                        286  ...   17770  1770   
E06000002            Middlesbrough                    2757  ...   23585  2400   
E06000003            Redcar and Cleveland              227  ...   27273  2893   
E06000004            Stockton-on-Tees                 1491  ...   42173  3632   
E06000005            Darlington                        468  ...   23803  2110   
...                                                    ...  ...     ...   ...   
NaN                  Torfaen                           231  ...   19396  2064   
                     Monmouthshire                     144  ...   29784  1865   
                     Newport                          2223  ...   3380